# Collect list of files with CCTP content and EBP reference

In [14]:
import pandas as pd
import re
import os 

from skrub import TableReport

pd.set_option('display.width', 500)

rep_drive_ATAE =  r"C:\Users\jch_m\ATAE"
rep_data_input =  r"C:\Users\jch_m\ATAE\Nicolas MONCEAU - NICOLAS\Stage_AppelsOffres\data_input"
rep_data_output = r"C:\Users\jch_m\ATAE\Nicolas MONCEAU - NICOLAS\Stage_AppelsOffres\data_output"

Extraction du contenu du log de script scannant les repertoires onedrive des CSPS
- num commande EBP => construction df_EBP
- filename de CCTP, CCAP, RC,  => construction df_consult
- fichiers rejets => df_rejet

In [ ]:
# Read and process le fichier log the script Script_listfiles.ps1

EBP = False
CONSULT = False
REJET = False

df2 = pd.DataFrame()
file_script_result = file_path = os.path.join(rep_data_input, "result_scan_AO_v6.txt")

with open(file_script_result, 'r', encoding='utf-8') as file:
    print(f"Contenu de '{file_script_result}'")
    for line_number, line in enumerate(file, 1):
        cur_line = line.strip()
        if "=== Liste ref EBP ===" in line:
            print("Processing des EBP")
            EBP = True
            continue
        elif "=== Liste fichiers CONSULT ===" in cur_line:
            #print("total EBP file:", nb_EBP_file, " Valid EBP:", nb_valid_EBP, "nb line read:",line_number)
            line_num_ebp = line_number
            print("Processing des fichiers de consultation")
            EBP = False
            CONSULT = True
            continue
        elif "=== Liste fichiers REJET ===" in cur_line:
            #print("total EBP file:", nb_EBP_file, " Valid EBP:", nb_valid_EBP, "nb line read:",line_number)
            line_num_ebp = line_number
            print("Processing des fichiers de REJET")
            EBP = False
            CONSULT = False
            REJET = True
            continue
        else:
            parts = line.split("\\jch_m\\ATAE\\") 
            #print(parts)
            if len(parts) > 1:
                line_segment = parts[1].split("\\")
                #print(line_segment)
                new_row_EBP = {0:''}
                if EBP:
                    new_row_EBP[0] = 'EBP'
                    for i in range(len(line_segment)):
                        new_row_EBP[i+1] = line_segment[i]
                    df2 = pd.concat([df2, pd.DataFrame([new_row_EBP])], ignore_index=True)
                if CONSULT:
                    new_row_EBP[0] = 'CONSULT'
                    for i in range(len(line_segment)):
                        new_row_EBP[i+1] = line_segment[i]
                    df2 = pd.concat([df2, pd.DataFrame([new_row_EBP])], ignore_index=True)
                if REJET:
                    new_row_EBP[0] = 'REJET'
                    for i in range(len(line_segment)):
                        new_row_EBP[i+1] = line_segment[i]
                    df2 = pd.concat([df2, pd.DataFrame([new_row_EBP])], ignore_index=True)

print("Shape du log scan df2",df2.shape)

if df2.shape[0] > 100_000:
    #rep_data_output = r"C:\DocPerso\Dev_Python\ProjetsPerso\BidManagement\data_output"
    path_scan_df2 = os.path.join(rep_data_output, "scan_df2.pkl")

    try:
        # Save the DataFrame to a pickle file
        df2.to_pickle(path_scan_df2)
        print(f"\nDataFrame successfully saved to '{path_scan_df2}'")
    except Exception as e:
        print(f"Error saving DataFrame: {e}")


Contenu de 'C:\Users\jch_m\ATAE\Nicolas MONCEAU - NICOLAS\Stage_AppelsOffres\data_input\result_scan_AO_v6.txt':
Processing des EBP
Processing des fichiers de consultation
Processing des fichiers de REJET
Shape du log scan df2 (149862, 16)

DataFrame successfully saved to 'C:\DocPerso\Dev_Python\ProjetsPerso\BidManagement\data_output\scan_df2.pkl'


In [15]:
##### NEW Version : EBP

df_EBP = pd.DataFrame()

df2_ebp = df2[df2[0] == "EBP"].copy() 

for i in range(len(df2_ebp)):
    ##### Extraction CSPS Name
    if isinstance(df2_ebp.iloc[i, 1], str):
        extracted_name = df2_ebp.iloc[i, 1]
    else:
        continue

    # Extraction EBP ID
    #print(df2_ebp[i][5],  df2_ebp[i][5].astype())
    if isinstance(df2_ebp.iloc[i, 5], str):
        match_numero = re.search(r"DE EBP (\d+)", df2_ebp.iloc[i, 5], re.IGNORECASE)
        numero_commande = match_numero.group(1) if match_numero else "Non trouvé"
    else:
        continue

    if numero_commande != "Non trouvé":
        # Extract EBP status
        #['04-ARCHIVES', '03-ARCHIVES', '01-CHANTIERS', '00-DEVIS', '03- ARCHIVES', '03- ARCHIVE', '03-ARCHIVE', '02-AO DEVIS A CHIFFRER', 'DOCS ATAE', 'commercial']
        directory_name = df2_ebp.iloc[i, 2]
        if "ARCHIVE" in directory_name:
            ebp_status = "Archive"
        elif "DEVIS" in directory_name:
            ebp_status = "Devis"
        elif "CHANTIER" in directory_name:
            ebp_status = "Chantier"
        else:
            continue # Passer à l'item suivant
            
        # Extraction ville Entreprise Mission
        match_details = re.search(r"([^\\]+) - ([^\\]+) - ([^\\]+)$", df2_ebp.iloc[i, 4])                
        if match_details:
            ville_code_dep = match_details.group(1)
            entreprise = match_details.group(2)
            mission_name = match_details.group(3)
        
            # Create a new row as a dictionary
            new_row_EBP = {
                'SPS Name': extracted_name.strip(),
                'ID EBP': numero_commande.strip(),
                'Ville': ville_code_dep.strip(),
                'Entreprise': entreprise.strip(),
                'Mission': mission_name.strip(),
                'statut': ebp_status,
            }
            df_EBP = pd.concat([df_EBP, pd.DataFrame([new_row_EBP])], ignore_index=True)
        else:
            #print("Détails EBP (ville, entreprise, rénovation) : Non trouvés")
            pass

print("Shape du df_EBP",df_EBP.shape)
path_df_EBP = os.path.join(rep_data_output, "df_EBP.pkl")

if df_EBP.shape[0]>2000:
    try:
        # Save the DataFrame to a pickle file
        df_EBP.to_pickle(path_df_EBP)
        print(f"\nDataFrame successfully saved to '{path_df_EBP}'")
    except Exception as e:
        print(f"Error saving DataFrame: {e}")

Shape du df_EBP (2633, 6)

DataFrame successfully saved to 'C:\Users\jch_m\ATAE\Nicolas MONCEAU - NICOLAS\Stage_AppelsOffres\data_output\df_EBP.pkl'


In [16]:
##### NEW Version : CONSULT
##### 

df_consult = pd.DataFrame()

df2_consult = df2[df2[0] == "CONSULT"].copy() 

for i in range(len(df2_consult)):
    ##### init
    numero_commande = ""
    ebp_status =  "not defined"
    
    ##### Extraction CSPS Name
    if isinstance(df2_consult.iloc[i, 1], str):
        extracted_name = df2_consult.iloc[i, 1]
        
        # re-create full_file path 
        valid_path_components = []
        current_row_series = df2_consult.iloc[i]
        for item in current_row_series:    
            if pd.isna(item) or item is None or item=="CONSULT":
                continue # Skip NaN or None values
            
            item_str = str(item) # Convert to string
            if item_str.lower() == 'nan' or not item_str.strip(): # Also skip string 'nan' or empty strings
                continue
            
            valid_path_components.append(item_str)

        part_file_path = os.path.join(*valid_path_components)
        file_path = os.path.join(rep_drive_ATAE, part_file_path)
        #print(f"Generated file_path: {file_path}\n")
        #if i > 3 :
        #    break
    
        #['04-ARCHIVES', '03-ARCHIVES', '01-CHANTIERS', '00-DEVIS', '03- ARCHIVES', '03- ARCHIVE', '03-ARCHIVE', '02-AO DEVIS A CHIFFRER', 'DOCS ATAE', 'commercial']    
        # init ebp status with same logic than EBP
        directory_name = df2_consult.iloc[i, 2]
        if "ARCHIVE" in directory_name:
            ebp_status = "Archive"
        elif "DEVIS" in directory_name:
            ebp_status = "Devis"
        elif "CHANTIER" in directory_name:
            ebp_status = "Chantier"
        else:
            continue # Passer à l'item suivant
        
        # then update ebp_status with Annule, Perdu , Non Repondu if applicable
        if "annul" in file_path.lower():
            ebp_status = "Annule"
        elif "echec" in file_path.lower() or "échec" in file_path.lower():
            ebp_status = "Perdu"
        elif "non repondu" in file_path.lower() or "non répondu" in file_path.lower():
            ebp_status = "Non repondu"

        # Extraction ville Entreprise Mission
        for j in range(4, 8): 
            current_cell_value = df2_consult.iloc[i, j]
            column_name = df2_consult.columns[j] # Get the actual column name for better printing
            if pd.isna(current_cell_value) or current_cell_value is None:
                continue # Move to the next column (j+1)
            string_to_search = str(current_cell_value)
            if string_to_search.lower() == 'nan' or not string_to_search.strip():
                continue # Move to the next column (j+1)
            
            match_details = re.search(r"([^\\]+) - ([^\\]+) - ([^\\]+)$", string_to_search)
            if not match_details:
                #print(f"    Skipping: No regex match found for '{string_to_search}'.")
                continue # Move to the next column (j+1)
            else:
                # If all checks pass, extract the groups
                ville_code_dep = match_details.group(1)
                entreprise = match_details.group(2)
                mission_name = match_details.group(3)

                # Create a new row as a dictionary
                new_row_EBP = {
                    'SPS Name': extracted_name.strip(),
                    'ID EBP': numero_commande.strip(),
                    'Ville': ville_code_dep.strip(),
                    'Entreprise': entreprise.strip(),
                    'Mission': mission_name.strip(),
                    'statut': ebp_status,
                    'file_path' : file_path.strip()
                }
                df_consult = pd.concat([df_consult, pd.DataFrame([new_row_EBP])], ignore_index=True)
        else:
            #print("Détails EBP (ville, entreprise, rénovation) : Non trouvés")
            pass
    else:
        continue


print("Shape du df_consult",df_consult.shape)
path_df_consult = os.path.join(rep_data_output, "df_consult.pkl")

if df_consult.shape[0]>100_000:
    try:
        # Save the DataFrame to a pickle file
        df_consult.to_pickle(path_df_consult)
        print(f"\nDataFrame successfully saved to '{path_df_consult}'")
    except Exception as e:
        print(f"Error saving DataFrame: {e}")

Shape du df_consult (147192, 7)

DataFrame successfully saved to 'C:\Users\jch_m\ATAE\Nicolas MONCEAU - NICOLAS\Stage_AppelsOffres\data_output\df_consult.pkl'


In [17]:
##### NEW Version : REJET
##### 

df_rejet = pd.DataFrame()

df2_rejet = df2[df2[0] == "REJET"].copy() 

for i in range(len(df2_rejet)):
    ##### Extraction CSPS Name
    if isinstance(df2_rejet.iloc[i, 1], str):
        extracted_name = df2_rejet.iloc[i, 1]
        
        # re-create full_file path 
        valid_path_components = []
        current_row_series = df2_rejet.iloc[i]
        for item in current_row_series:    
            if pd.isna(item) or item is None or item=="REJET":
                continue # Skip NaN or None values
            
            item_str = str(item) # Convert to string
            if item_str.lower() == 'nan' or not item_str.strip(): # Also skip string 'nan' or empty strings
                continue
            
            valid_path_components.append(item_str)

        part_file_path = os.path.join(*valid_path_components)
        file_path = os.path.join(rep_drive_ATAE, part_file_path)
        #print(f"Generated file_path: {file_path}\n")
        #if i > 3 :
        #    break
        
        # init ebp_status et ebp_id  
        ebp_status = "Perdu"
        ebp_id = "not defined"

        # Extraction ville Entreprise Mission
        for j in range(4, 8): 
            current_cell_value = df2_rejet.iloc[i, j]
            if pd.isna(current_cell_value) or current_cell_value is None:
                continue # Move to the next column (j+1)
            string_to_search = str(current_cell_value)
            if string_to_search.lower() == 'nan' or not string_to_search.strip():
                continue # Move to the next column (j+1)
            
            match_details = re.search(r"([^\\]+) - ([^\\]+) - ([^\\]+)$", string_to_search)
            if not match_details:
                continue # Move to the next column (j+1)
            else:
                # If all checks pass, extract the groups
                ville_code_dep = match_details.group(1)
                entreprise = match_details.group(2)
                mission_name = match_details.group(3)

                # Create a new row as a dictionary
                new_row_EBP = {
                    'SPS Name': extracted_name.strip(),
                    'ID EBP': ebp_id,
                    'Ville': ville_code_dep.strip(),
                    'Entreprise': entreprise.strip(),
                    'Mission': mission_name.strip(),
                    'statut': ebp_status,
                    'file_path' : file_path
                }
                df_rejet = pd.concat([df_rejet, pd.DataFrame([new_row_EBP])], ignore_index=True)
        else:
            #print("Détails EBP (ville, entreprise, rénovation) : Non trouvés")
            pass
    else:
        continue

print("Shape du df_rejet",df_rejet.shape)
path_df_rejet = os.path.join(rep_data_output, "df_rejet.pkl")

if df_rejet.shape[0]>50:
    try:
        # Save the DataFrame to a pickle file
        df_rejet.to_pickle(path_df_rejet)
        print(f"\nDataFrame successfully saved to '{path_df_rejet}'")
    except Exception as e:
        print(f"Error saving DataFrame: {e}")

Shape du df_rejet (172, 7)

DataFrame successfully saved to 'C:\Users\jch_m\ATAE\Nicolas MONCEAU - NICOLAS\Stage_AppelsOffres\data_output\df_rejet.pkl'


matched_df = pd.merge(df_EBP, df_consult, on=match_columns, how='inner', suffixes=('_ebp', '_consult'))
    there are  433  EBP ID matching
    Taille du fichier consolidé (ID EBP/Consult):1518, Taille du fichier EBP:3151, Taille du fichier consult:2027
    Nb unique EBP ID dans fichier consolidé:433, Nb unique EBP ID dans le fichier EBP:2685 

    

In [18]:
# Merge df_consult and df_EBP pour sync EBP ID on the mission/files
# note: no need to sync EBP with rejet, because none of the Rejet file match an EBP entry (dans la liste des repertoires et fichiers). normal ?

# Define the columns to match on
match_columns = ['SPS Name', 'Ville', 'Entreprise', 'Mission']

df_consult_ebp = pd.merge(df_consult, df_EBP, on=match_columns, how='left', suffixes=('_consult', '_ebp'))
df_consult_ebp['ID EBP'] = df_consult_ebp['ID EBP_ebp'].fillna("no EBP")
df_consult_ebp = df_consult_ebp.drop(columns=['ID EBP_ebp','ID EBP_consult', 'statut_ebp'])


In [19]:
TableReport(df_consult_ebp)

Processing column   7 / 7


,,,,,,,


In [20]:
TableReport(df_rejet)

Processing column   7 / 7


,,,,,,,
